In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv("./new_preprocessed_final.csv")

In [3]:
# TF‑IDF + token overlap
tfidf_feats = [c for c in df.columns if ("cosine" in c.lower() and "bert" not in c.lower()) or "overlap" in c.lower()]

# BERT similarity
bert_feat = None
for c in df.columns:
  if "bert_similarity" in c.lower() or ("cosine" in c.lower() and "bert" in c.lower()):
    bert_feat = c
    break

# Extra linguistic & stylistic features
extra_feats = ["title_length","sensational_word_count","punctuation_count","punctuation_ratio"]
extra_feats = [c for c in extra_feats if c in df.columns]

# Combine all features
feature_cols = tfidf_feats + ([bert_feat] if bert_feat else []) + extra_feats

# Label column
label_col = next(c for c in df.columns if "is_clickbait" in c.lower())

# Data
X = df[feature_cols]
y = df[label_col]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=42, stratify=y
)

# Perceptron pipeline
pipe = Pipeline([
  ("scaler", StandardScaler()),
  ("clf", Perceptron(max_iter=1000, tol=1e-3, class_weight="balanced", random_state=42))
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print("=== Features Used ===")
print(feature_cols)

print("\n=== Classification Report (SLP + ALL features) ===")
print(classification_report(y_test, y_pred))

print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

=== Features Used ===
['cosine_similarity', 'token_overlap', 'bert_similarity', 'title_length', 'sensational_word_count', 'punctuation_count', 'punctuation_ratio']

=== Classification Report (SLP + ALL features) ===
              precision    recall  f1-score   support

           0       0.99      0.89      0.94       514
           1       0.59      0.92      0.72        86

    accuracy                           0.90       600
   macro avg       0.79      0.91      0.83       600
weighted avg       0.93      0.90      0.91       600


=== Confusion Matrix ===
[[460  54]
 [  7  79]]
